In [ ]:
import os
import shutil # Added for moving files
from collections import defaultdict
from typing import List, Tuple, Dict

def get_folder_name_from_filename(filename: str) -> str:
    """
    Extracts the 'YYYYMMDD_HHMMSS' part from a filename like
    'camX_YYYYMMDD_HHMMSS_resolution.mp4'.
    """
    base_name = os.path.basename(filename)
    # Remove 'camX_' prefix (e.g., 'cam0_', 'cam1_')
    if base_name.startswith("cam0_") or base_name.startswith("cam1_"):
        name_part = base_name.split("_", 1)[1] # Get part after first underscore
    else:
        # Fallback or error if naming convention is unexpected
        # For now, assume it's the identifier part directly if no prefix
        name_part = base_name

    # Split the remaining part by underscores
    # Expected: YYYYMMDD_HHMMSS_resolution.mp4
    identifier_parts = name_part.split("_")
    if len(identifier_parts) >= 2:
        return f"{identifier_parts[0]}_{identifier_parts[1]}"
    else:
        # Fallback if the structure is not as expected,
        # use the part before the first dot (extension)
        return name_part.rsplit('.', 1)[0]


def pair_camera_files_and_move(directory_path: str) -> None:
    """
    Pairs 'cam0' and 'cam1' files from the specified directory,
    creates a new subfolder for each pair based on their timestamp,
    and moves the paired files into their respective new subfolder.

    Args:
        directory_path: The path to the directory containing the video files
                        and where new subfolders will be created.

    Raises:
        FileNotFoundError: If the specified directory_path does not exist
                           or is not a directory.
    """
    if not os.path.isdir(directory_path):
        raise FileNotFoundError(f"Error: The directory '{directory_path}' was not found or is not a directory.")

    files_in_directory = os.listdir(directory_path)
    # Using a dictionary to group files by a common identifier (timestamp_resolution)
    # Key: YYYYMMDD_HHMMSS_resolution.mp4 -> Value: {'cam0': path, 'cam1': path}
    file_groups: Dict[str, Dict[str, str]] = defaultdict(lambda: {"cam0": "", "cam1": ""})

    for filename in files_in_directory:
        full_file_path = os.path.join(directory_path, filename)
        if os.path.isfile(full_file_path): # Process only files
            if filename.endswith(".mp4") and (filename.startswith("cam0_") or filename.startswith("cam1_")):
                try:
                    parts = filename.split("_", 1) # cam0, YYYYMMDD_HHMMSS_resolution.mp4
                    camera_type = parts[0]
                    common_identifier = parts[1] # YYYYMMDD_HHMMSS_resolution.mp4

                    if camera_type == "cam0":
                        file_groups[common_identifier]["cam0"] = full_file_path
                    elif camera_type == "cam1":
                        file_groups[common_identifier]["cam1"] = full_file_path
                except IndexError:
                    print(f"Warning: Could not parse filename '{filename}' according to expected format 'camX_identifier.mp4'. Skipping.")
                    continue


    moved_pairs_count = 0
    if not file_groups:
        print("No video files found matching 'cam0_' or 'cam1_' prefixes.")
        return

    for common_id, cams in file_groups.items():
        cam0_file = cams["cam0"]
        cam1_file = cams["cam1"]

        if cam0_file and cam1_file: # Both files exist for this common_id
            # Determine the new folder name (e.g., "20250523_101042")
            # We can use either cam0_file or cam1_file's basename for this
            folder_name_base = get_folder_name_from_filename(os.path.basename(cam0_file))
            new_folder_path = os.path.join(directory_path, folder_name_base)

            try:
                os.makedirs(new_folder_path, exist_ok=True) # Create the folder

                # Move cam0 file
                destination_cam0 = os.path.join(new_folder_path, os.path.basename(cam0_file))
                shutil.move(cam0_file, destination_cam0)
                print(f"Moved: {os.path.basename(cam0_file)} -> {os.path.relpath(destination_cam0, directory_path)}")

                # Move cam1 file
                destination_cam1 = os.path.join(new_folder_path, os.path.basename(cam1_file))
                shutil.move(cam1_file, destination_cam1)
                print(f"Moved: {os.path.basename(cam1_file)} -> {os.path.relpath(destination_cam1, directory_path)}")

                moved_pairs_count += 1
            except Exception as e:
                print(f"Error processing pair for ID '{common_id}' (Folder: {folder_name_base}): {e}")
                # Potentially move files back if one succeeded and the other failed,
                # or add more sophisticated error handling if needed.
        elif cam0_file or cam1_file:
            # Only one of the pair was found for this common_id
            unpaired_file = cam0_file or cam1_file
            print(f"Info: Found unpaired file: {os.path.basename(unpaired_file)} for identifier '{common_id}'. It will not be moved.")


    if moved_pairs_count == 0 and any(file_groups.values()): # Files were found but no complete pairs
        print("\nFound video files, but no complete cam0/cam1 pairs were formed to be moved.")
    elif moved_pairs_count > 0:
        print(f"\nSuccessfully moved {moved_pairs_count} pair(s) into respective folders.")
    else: # This else implies no file_groups were populated initially.
        pass # Initial message "No video files found..." already covers this.


if __name__ == "__main__":
    try:
        # Use the current working directory (directory of the notebook/script)
        current_directory = os.getcwd()
    except NameError: # Fallback if __file__ isn't defined (less common for os.getcwd())
        current_directory = "." # Represents current directory

    print(f"Script running in: '{os.path.abspath(current_directory)}'")
    print(f"Processing video files and organizing them into subfolders within this directory...")

    try:
        pair_camera_files_and_move(current_directory)
    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"An unexpected error occurred during the process: {e}")